### 기본 세팅

In [1]:
import time
import pandas as pd
import tqdm
import pickle

from selenium import webdriver
import pyperclip
import klembord
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import multiprocessing
from multiprocessing import Pool

import clipboard



### 네이버 카페 로그인

In [2]:
def copy_input(xpath, input):
    #pyperclip.copy(input)
    #klembord.set_text(input)
    clipboard.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

In [6]:
op = webdriver.ChromeOptions()
op.add_argument('--headless')
driver = webdriver.Chrome('/home/super/sr_work/chromedriver',options=op)
driver.implicitly_wait(3)

In [7]:
naver_id = 'ksl970330'
naver_pw = 'zz43574357!'

driver.get('https://nid.naver.com/nidlogin.login')             
time.sleep(1)

copy_input('//*[@id="id"]', naver_id)
time.sleep(1)
copy_input('//*[@id="pw"]', naver_pw)
time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

PyperclipException: 
    Pyperclip could not find a copy/paste mechanism for your system.
    For more information, please visit https://pyperclip.readthedocs.io/en/latest/index.html#not-implemented-error 

### 크롤링

In [5]:
def crawling_address_and_nickname(menuid):
    page = 0
    address = []
    nickname = []

    while page <= 1:
        try:
            page += 1
            driver.get(f"https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.menuid={menuid}&search.boardtype=L&search.page={page}")
            driver.switch_to.frame("cafe_main")

            try:
                if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
                    break
            except:
                pass

            for i in range(1, 51, 1):
                cur_address = driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]").get_attribute("href")
                cur_nickname = driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[2]/div/table/tbody/tr/td/a").get_attribute("onclick")
                
                address.append(cur_address)
                nickname.append(cur_nickname)

            driver.switch_to.default_content()   
            break
        except:
            pass
        
    return address, nickname

In [6]:
# nickname에 mbti가 있으면 추출
def find_mbti_in_nickname(nickname, mbti_group):
    for i in range(len(nickname)):
        nickname[i] = nickname[i].lower()
        
    drop_idx = []   

    for i in range(len(nickname)):
        check_cnt = 0

        for mbti in mbti_group:        
            if mbti in nickname[i]:
                nickname[i] = mbti

            if mbti not in nickname[i]:
                check_cnt += 1

        if check_cnt == 16:
            drop_idx.append(i)
            
    return nickname, drop_idx

In [7]:
# nickname에 mbti가 없으면 드랍
def drop_idx_in_address_and_nickname(address, nickname, drop_idx):
    for i in range(len(address)):
        while i in drop_idx:
            address.pop(i)
            nickname.pop(i)
            drop_idx.remove(i)

            for idx in range(len(drop_idx)):
                drop_idx[idx] -= 1
                
    return address, nickname

In [8]:
def crawling_title_and_content(address):
    title = []
    content = []
    
    for url in tqdm(address):
        try:
            driver.get(url)
            driver.switch_to.frame("cafe_main")
            
            cur_title = driver.find_element_by_css_selector("h3.title_text").text
            cur_content = driver.find_element_by_class_name("se-main-container").text

            title.append(cur_title)
            content.append(cur_content)

        except:
            pass
    
    return title, content

In [9]:
def crawling(menuid, mbti_group):
    pool = Pool(multiprocessing.cpu_count() * 2)
    
    print(' crawling address and nickname... ')
    start_time = time.time()
    address, nickname = crawling_address_and_nickname(menuid) 
    print('  progress time: {:.7f} seconds \n'.format(time.time() - start_time))
    
    
    print(' extracting mbti from nickname... ')
    start_time = time.time()
    nickname, drop_idx = find_mbti_in_nickname(nickname, mbti_group)                    # nickname에 mbti가 있으면 추출
    address, nickname = drop_idx_in_address_and_nickname(address, nickname, drop_idx)   # nickname에 mbti가 없으면 드랍
    print('  progress time: {:.7f} seconds \n'.format(time.time() - start_time))
    
    
    print(' crawling title and content... ')
    start_time = time.time()
    results = pool.map(crawling_title_and_content, address) 
    pool.close()
    pool.join()
    
    title = []
    nickname = []
    for result in results:
        title.append(result[0])
        nickname.append(result[1])
    print('  progress time: {:.7f} seconds \n'.format(time.time() - start_time))
    
    
    data = pd.DataFrame({'mbti': nickname, 'title': title, 'content': content})
    return data

In [10]:
mbti_group = ['intj', 'intp', 'entj', 'entp',
              'infj', 'infp', 'enfj', 'enfp',
              'istj', 'isfj', 'estj', 'esfj',
              'istp', 'isfp', 'estp', 'esfp']

In [ ]:
# INFP & ENFP 게시판 크롤링, menuid=18
data18 = crawling(18, mbti_group)
data18.head(5)

 crawling address and nickname... 
  progress time: 15.8801427 seconds 

 extracting mbti from nickname... 
  progress time: 0.0000000 seconds 

 crawling title and content... 


In [25]:
data18.to_pickle('menuid=18 crawling.pkl')

with open("./menuid=18 crawling.pickle", "wb") as MyFile:
    pickle.dump(data18, MyFile)

In [13]:
# INTP & ENTP 게시판 크롤링, menuid=17
data17 = crawling(17, mbti_group)
data17.head(5)

,mbti,title,content
0,entp,엔팁이 하는 30문답!,Q1. 사소한데 정 떨어지는 행동은?\n- 손톱 밑 때 같은 자기 몸이나 물건 더러...
1,intp,내로남불? 내불남로!,인팁이 멍청한 사람 싫어한다는데 저는 저 한정인 것 같아요😂 같은 실수 반복하면 스...
2,intp,인팁이 이렇게 단순합니다,제가 맞춤법돌리는거 귀차나서 걍 글귀만 쓰는데 이거보고 기분좋아져서 헤실헤실거리는중...
3,intj,인팁분들 맞춤법,신경 많이 쓰시나요? 오타 살짝나도 바로바로 고쳐서 보내는데 본인이 편하니까 고치는...
4,intp,라세님의 30문답,심심해서 해보는 30문답!\nQ1. 사소한데 정 떨어지는 행동은?\n- 맨날 이거싫...


In [14]:
data17.to_pickle('menuid=17 crawling.pkl')

with open("./data17.pickle", "wb") as MyFile:
    pickle.dump(data17, MyFile)

In [15]:
# ISFP & ESFP 게시판 크롤링, menuid=16
data16 = crawling(16, mbti_group)
data16.head(5)

,mbti,title,content
0,enfp,5개월간 본 isfp 특징,*짝사랑했던 상대라 조금 다르게 볼 수 있음 주의*\n1.아닌건 진짜 아니다. 근데...
1,isfp,새벽의 인간관계 30문 30답,Q1. 사소한데 정 떨어지는 행동은?\n- 맞춤법 심각하게 틀릴 때 \nQ2. 사람...
2,intp,잇프피 호감표시??,잇프피에게 연락해보고 영 아니다 싶으면 끊으려고 하는데\n잇프피 남자님들은 호감 있...
3,enfj,ISFP 남자가 단답을 너무 자주 써요,다름이 아닌 제 남자친구인 ISFP의 이야기인데\n어느 순간부터 (예상 1달 반 전...
4,isfp,사람은 왜 열심히 살아야 할까요?,일 잘하고 싶고 열정많은 사람들이 부럽고 신기해요.\n전 그냥 놀고 먹고 싶은데. ...


In [16]:
data16.to_pickle('menuid=16 crawling.pkl')

with open("./data16.pickle", "wb") as MyFile:
    pickle.dump(data16, MyFile)

In [17]:
# ISTP & ESTP 게시판 크롤링, menuid=15
data15 = crawling(15, mbti_group)
data15.head(5)

,mbti,title,content
0,isfp,엣팁분들!!!!!,원래 엉이나 웅같은 거 잘 쓰나요?? 평소에 어라고 쓰던 애가 저런 거 쓰니까 넘 ...
1,infp,잇팁 남자 마음이 뭔지 모르겠어요,갠적인 연락은 거의 안하는데 \n나가 놀고싶은데 더워서 못나가겠다하니 빙수 기프티콘...
2,entj,istp 와 entj,istp 남자 entj 여자 친구로써 궁합 어떤가요?
3,entj,estp 친구,estp 동성 친구가 자꾸 카톡을 보내는데 친해지고 싶어서 그런걸까요?
4,istp,"설거지,,",설거지 해야되는데 하고\n두시간째 폰보면서 누워있네요\n분명 설거지하려고 맘 먹었는...


In [18]:
data15.to_pickle('menuid=15 crawling.pkl')

with open("./data15.pickle", "wb") as MyFile:
    pickle.dump(data15, MyFile)

In [19]:
# INFJ & ENFJ 게시판 크롤링, menuid=14
data14 = crawling(14, mbti_group)
data14.head(5)

,mbti,title,content
0,infj,심심한 밤 인프제 남 질문받아요~ 다시 돌아온 상담소,안녕 칭구들!! 댕댕이 아조씨야!
1,istp,ENFJ가 부모가되면 어떤 성격인가요?,ENFJ유형이 부모가 되면 어떤 성격이실지궁금해요 \n예상이나 상상도 괜찮습니당 내...
2,infp,Infj 거리 둘 마음이면 뭐 안받겠죠?,별 일은 없는데\n저혼자 괜히 신경쓰이는 게 있어서 (불안형이라 ㅠ)\n혹시 인프제...
3,infp,INFJ가 되고싶습니다!,INFJ가 되고 싶은 INFP입니다!\nINFJ의 성숙함과 성실함이 너무 보기 좋아...
4,infj,인프제 & 엔팁이 궁합최고인 이유,( 엔팁 남편과 궁합이 너무 잘 맞는\n인프제 아내가 쓰는 글 😉💙)\n**상당히 ...


In [20]:
data14.to_pickle('menuid=14 crawling.pkl')

with open("./data14.pickle", "wb") as MyFile:
    pickle.dump(data14, MyFile)

In [21]:
# ISFJ & ESFJ 게시판 크롤링, menuid=13
data13 = crawling(13, mbti_group)
data13.head(5)

,mbti,title,content
0,istp,이사람 isfj 인데 이사람이 ㄸㄹㅇ인가요 아님 특징인가요,하나라도 손해보기 싫어하고 자기가 손해인지 아닌지 \n하나하나 계산해봄 (이과정이더...
1,esfj,당일치기로 서울 다녀오기,김포 갈때\n다시 김해로
2,entp,잇프제 아내가 좋아하는 말,"여자친구와는 대학교 CC로 만나 일주일에 6일을 만났고, 별일 없으면 주말에는 하루..."
3,esfj,엣프제 질문받을게요:),이런거 처음 해보네요!\n정식검사로 나온 찐 엣프제입니당\n궁금하신점 있으시담 질문...
4,entp,잇프제분들 질문있어여!!!!!!,저 댓글달다가 너무너무 궁금해져서 달려왔자나여 \n저 엔팁이라 왜살인마라서 ㅋㅋㅋㅋ...


In [22]:
data13.to_pickle('menuid=13 crawling.pkl')

with open("./data13.pickle", "wb") as MyFile:
    pickle.dump(data13, MyFile)

In [23]:
# INTJ & ENTJ 게시판 크롤링, menuid=12
data12 = crawling(12, mbti_group)
data12.head(5)

,mbti,title,content
0,intj,이별 기념 INTJ 질문받아요 ㅋㅋ,꽁하고 땅굴 팔바에 그냥 공부하고 다른 걸 하면서 시간 떼우는게 멘탈회복에 좋다는 ...
1,entj,ENTJ 8w7 알려주실분,카페에서 놀다가 8w7 이런 식의 닉네임이 있길래 알아보다보니 애니어그램이더라구요....
2,entj,엔티제 자기소개,계획적 + 추진력좋음 = 바로 실행가능한 계획을 많이세움 (시뮬돌려보구 터무니없음 ...
3,entj,ENTJ가 이성에게 호감을 갖게 되는 몇가지 단계들,글에 앞서 분명하게 밝힐것은\n당연히 사람마다 다 다르다는것이다\n나는 이 아래의 ...
4,intj,하루종일 수많은 생각에 둥둥 떠다니는 느낌(INTJ),정말정말 사소한 건데도 심각할 정도로 분석하고 따지게 돼요.\nA를 할까 B를 할까...


In [24]:
data12.to_pickle('menuid=12 crawling.pkl')

with open("./data12.pickle", "wb") as MyFile:
    pickle.dump(data12, MyFile)

In [25]:
# ISTJ & ESTJ 게시판 크롤링, menuid=11
data11 = crawling(11, mbti_group)
data11.head(5)

,mbti,title,content
0,isfj,istj 여성분을 좋아하는데 이 분이 무슨 마음인걸까요?,"전반적인 상황은\n- 상대방 istj, 대화, 카톡 등등 감정기복 거의 없는 로보..."
1,enfp,ESTJ 남자한테 꽃?을 받았는데..,"동네 모임에서 알게된 남자고 여러명이서 두번정도 그냥 인사정도만하고,\n둘이 그냥 ..."
2,istp,연인간 결혼얘기하면 짜증나는 분,"네 접니다,, \n그동안 남친들이 ""우리 나중에 애 몇명 낳고~ 어디 살고~""\n이..."
3,estj,인간 관계를 포기하게돼요,"예전에는 누구랑 말싸움하는 상황이 되면, 무조건 내 말 인정하게 납득시켰어요.\n어..."
4,enfp,잇티제가 오래 본 이성을 선호하는 이유,잇티제 관련 글을 보다 보면 사귀기 전에 이성을 오랫동안 본다는 글이 많더라구요!!...


In [26]:
data11.to_pickle('menuid=11 crawling.pkl')

with open("./data11.pickle", "wb") as MyFile:
    pickle.dump(data11, MyFile)